In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("vard2.csv")

In [3]:
df['var_salary'] = df['var_salary'].map(lambda x:int(x))

In [4]:
df.head()

,id_job,var_department,var_telecommuting,var_logo,var_questions,var_employment_type,var_required_experience,var_required_education,var_industry,var_function,var_pais,var_salary
0,0,Marketing,0,1,0,Other,Other,Unspecified,Unknown,Marketing,United States,0
1,1,Other,0,1,0,Full-time,Not Applicable,Unspecified,Other,Customer Service,Other,0
2,2,Unknown,0,1,0,Unknown,Unknown,Unspecified,Unknown,Unknown,United States,0
3,3,Sales,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,United States,0
4,4,Unknown,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Other,Other,United States,0


In [5]:
aux = pd.read_csv("fake_job_postings.csv")

In [6]:
aux['job_id'] = aux['job_id']-1

In [7]:
aux = aux[['job_id','fraudulent']]

In [8]:
aux.rename(columns={'job_id':'id_job'},inplace=True)

In [9]:
aux

,id_job,fraudulent
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
17875,17875,0
17876,17876,0
17877,17877,0
17878,17878,0


In [10]:
df = df.merge(aux)

In [11]:
df.shape

(17880, 13)

In [12]:
df.drop(columns='id_job')

,var_department,var_telecommuting,var_logo,var_questions,var_employment_type,var_required_experience,var_required_education,var_industry,var_function,var_pais,var_salary,fraudulent
0,Marketing,0,1,0,Other,Other,Unspecified,Unknown,Marketing,United States,0,0
1,Other,0,1,0,Full-time,Not Applicable,Unspecified,Other,Customer Service,Other,0,0
2,Unknown,0,1,0,Unknown,Unknown,Unspecified,Unknown,Unknown,United States,0,0
3,Sales,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,United States,0,0
4,Unknown,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Other,Other,United States,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
17875,Sales,0,1,1,Full-time,Mid-Senior level,Unspecified,Computer Software,Sales,Other,0,0
17876,Other,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Other,Other,United States,0,0
17877,Unknown,0,0,0,Full-time,Unknown,Unspecified,Unknown,Unknown,United States,0,0
17878,Unknown,0,0,1,Contract,Not Applicable,Other,Other,Other,Other,0,0


In [13]:
df.shape

(17880, 13)

In [14]:
df.rename(columns={'fraudulent':'y'},inplace = True)

In [15]:
df.drop(columns='id_job')

,var_department,var_telecommuting,var_logo,var_questions,var_employment_type,var_required_experience,var_required_education,var_industry,var_function,var_pais,var_salary,y
0,Marketing,0,1,0,Other,Other,Unspecified,Unknown,Marketing,United States,0,0
1,Other,0,1,0,Full-time,Not Applicable,Unspecified,Other,Customer Service,Other,0,0
2,Unknown,0,1,0,Unknown,Unknown,Unspecified,Unknown,Unknown,United States,0,0
3,Sales,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,United States,0,0
4,Unknown,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Other,Other,United States,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
17875,Sales,0,1,1,Full-time,Mid-Senior level,Unspecified,Computer Software,Sales,Other,0,0
17876,Other,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Other,Other,United States,0,0
17877,Unknown,0,0,0,Full-time,Unknown,Unspecified,Unknown,Unknown,United States,0,0
17878,Unknown,0,0,1,Contract,Not Applicable,Other,Other,Other,Other,0,0


## Transformación WoE y cálculo de IV 

In [16]:
varw = [v for v in df.columns if v[:3]=='var']

In [17]:
varw

['var_department',
 'var_telecommuting',
 'var_logo',
 'var_questions',
 'var_employment_type',
 'var_required_experience',
 'var_required_education',
 'var_industry',
 'var_function',
 'var_pais',
 'var_salary']

In [18]:
l_iv = []
for v in varw:
    aux = df[[v,'y']].copy()
    aux['n'] = 1
    aux = aux.pivot_table(index=v,columns='y',values='n',aggfunc='sum',fill_value=0)
    for k in range(2):
        aux[k]/=aux[k].sum() 
    aux['woe'] = np.log(aux[0]/aux[1])
    iv = ((aux[0]-aux[1])*aux['woe']).sum()
    l_iv.append((v,iv))
    aux.drop(range(2),axis=1,inplace=True)
    df['w_%s'%v]=df[v].replace(aux.to_dict()['woe'])

In [19]:
iv = pd.DataFrame(l_iv,columns=['característica','iv']).sort_values('iv',ascending=False).reset_index(drop=True)

In [20]:
iv

,característica,iv
0,var_logo,1.099602
1,var_pais,0.392413
2,var_function,0.360372
3,var_industry,0.281461
4,var_required_education,0.260127
5,var_questions,0.194173
6,var_required_experience,0.179778
7,var_salary,0.133991
8,var_employment_type,0.118899
9,var_department,0.092371


In [21]:
best = iv.loc[(iv.iv>=0.1)]['característica'].to_list()

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,roc_auc_score

In [23]:
modelo = LogisticRegression()

In [24]:
varw = ['w_%s'%v for v in best]
X = df[varw].copy()
y = df['y'].copy()

In [25]:
Xt, Xv, yt,yv = train_test_split(X,y,train_size=0.7)

In [26]:
modelo.fit(Xt,yt)

/home/diegoias/Documents/CieciaDatos/cdd/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [27]:
print(roc_auc_score(y_true=yt,y_score=modelo.predict_proba(Xt)[:,1]))
print(roc_auc_score(y_true=yv,y_score=modelo.predict_proba(Xv)[:,1]))

0.8562049750143544
0.8563248586004214


In [28]:
print(accuracy_score(y_true=yt,y_pred=modelo.predict(Xt)))
print(accuracy_score(y_true=yv,y_pred=modelo.predict(Xv)))

0.9551773729626079
0.9498508575689784


In [29]:
modelo.coef_

array([[-0.86849093, -0.86875847, -0.92107308, -0.6335976 , -0.47092419,
        -0.02120582, -0.20021421, -1.31395854, -0.68114026]])

In [30]:
for v in best:
    aux = df[[v,'y']].copy()
    aux['n'] = 1
    aux = aux.pivot_table(index=v,columns='y',values='n',aggfunc='sum',fill_value=0)
    aux['t'] = aux[range(2)].sum(axis=1)
    for k in range(2):
        aux[k]/=aux['t']
    print(v)
    print('\n')
    print(aux)
    print('\n')

var_logo


y                0         1      t
var_logo                           
0         0.840710  0.159290   3660
1         0.980098  0.019902  14220


var_pais


y                      0         1      t
var_pais                                 
Other           0.976653  0.023347   4840
United Kingdom  0.990352  0.009648   2384
United States   0.931494  0.068506  10656


var_function


y                              0         1     t
var_function                                    
Administrative          0.811111  0.188889   630
Customer Service        0.945484  0.054516  1229
Engineering             0.916172  0.083828  1348
Information Technology  0.981704  0.018296  1749
Marketing               0.987952  0.012048   830
Other                   0.964757  0.035243  4171
Sales                   0.972071  0.027929  1468
Unknown                 0.947792  0.052208  6455


var_industry


y                                           0         1     t
var_industry                        